# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [9]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [11]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [14]:
bob._objects

{99731784642: tensor([1, 2, 3, 4, 5])}

In [15]:
x.location

<VirtualWorker id:bob #objects:1>

In [16]:
x.id_at_location

99731784642

In [17]:
x.id

80204050205

In [18]:
x.owner

<VirtualWorker id:me #objects:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [20]:
x

(Wrapper)>[PointerTensor | me:80204050205 -> bob:99731784642]

In [21]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x= th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [26]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [28]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [44]:
x

(Wrapper)>[PointerTensor | me:84248250794 -> bob:64032220097]

In [45]:
y

(Wrapper)>[PointerTensor | me:66658247205 -> bob:62920035025]

In [0]:
z = x + y

In [47]:
z

(Wrapper)>[PointerTensor | me:7549089470 -> bob:99286676007]

In [48]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:50535594766 -> bob:51540055303]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [38]:
z.backward()

(Wrapper)>[PointerTensor | me:73643338848 -> bob:22775581503]

In [0]:
x = x.get()

In [40]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [55]:
for i in range(10) :
  
     pred = input.mm(weights)
      
     loss = ((pred - target)**2).sum()
    
     loss.backward()
      
     weights.data.sub_(weights.grad * 0.1)
     weights.grad *= 0
      
     print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [57]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
bob._objects

{55915786223: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [61]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{21618928272: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [65]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [67]:
x

(Wrapper)>[PointerTensor | me:36775045188 -> bob:37172090972]

In [68]:
bob._objects

{37172090972: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [70]:
bob._objects

{37172090972: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [72]:
bob._objects

{37172090972: tensor([1, 2, 3, 4, 5])}

In [73]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [75]:
bob._objects

{78064462313: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [77]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [82]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(4.7627)
tensor(1.8633)
tensor(1.1185)
tensor(0.7219)
tensor(0.4714)
tensor(0.3092)
tensor(0.2036)
tensor(0.1347)
tensor(0.0896)
tensor(0.0599)
tensor(0.0403)
tensor(0.0273)
tensor(0.0186)
tensor(0.0128)
tensor(0.0089)
tensor(0.0062)
tensor(0.0044)
tensor(0.0031)
tensor(0.0022)
tensor(0.0016)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [87]:
train()

tensor(0.5232, requires_grad=True)
tensor(0.3466, requires_grad=True)
tensor(0.1689, requires_grad=True)
tensor(0.2104, requires_grad=True)
tensor(0.0933, requires_grad=True)
tensor(0.1226, requires_grad=True)
tensor(0.0537, requires_grad=True)
tensor(0.0713, requires_grad=True)
tensor(0.0311, requires_grad=True)
tensor(0.0415, requires_grad=True)
tensor(0.0180, requires_grad=True)
tensor(0.0242, requires_grad=True)
tensor(0.0104, requires_grad=True)
tensor(0.0141, requires_grad=True)
tensor(0.0060, requires_grad=True)
tensor(0.0082, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0001, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [90]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [93]:
bob._objects

{37859075752: tensor([1, 2, 3, 4, 5])}

In [94]:
alice._objects

{58101277340: (Wrapper)>[PointerTensor | alice:58101277340 -> bob:37859075752]}

In [0]:
y = x + x

In [96]:
y

(Wrapper)>[PointerTensor | me:88129808966 -> alice:84122619342]

In [97]:
bob._objects

{24456453897: tensor([ 2,  4,  6,  8, 10]),
 37859075752: tensor([1, 2, 3, 4, 5])}

In [98]:
alice._objects

{58101277340: (Wrapper)>[PointerTensor | alice:58101277340 -> bob:37859075752],
 84122619342: (Wrapper)>[PointerTensor | alice:84122619342 -> bob:24456453897]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [101]:
bob._objects

{34176644655: tensor([1, 2, 3, 4, 5])}

In [102]:
alice._objects

{40148693556: (Wrapper)>[PointerTensor | alice:40148693556 -> bob:34176644655]}

In [103]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40148693556 -> bob:34176644655]

In [104]:
bob._objects

{34176644655: tensor([1, 2, 3, 4, 5])}

In [105]:
alice._objects

{}

In [106]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [107]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [109]:
bob._objects

{13352301478: tensor([1, 2, 3, 4, 5])}

In [110]:
alice._objects

{35572827053: (Wrapper)>[PointerTensor | alice:35572827053 -> bob:13352301478]}

In [0]:
del x

In [112]:
bob._objects

{}

In [113]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [114]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [116]:
bob._objects

{68235688951: tensor([1, 2, 3, 4, 5])}

In [117]:
alice._objects

{}

In [118]:
x.move(alice)

(Wrapper)>[PointerTensor | me:41477165916 -> alice:39296539549]

In [119]:
bob._objects

{}

In [120]:
alice._objects

{39296539549: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [122]:
bob._objects

{50519296812: tensor([1, 2, 3, 4, 5])}

In [123]:
alice._objects

{39296539549: tensor([1, 2, 3, 4, 5]),
 69575318925: (Wrapper)>[PointerTensor | alice:69575318925 -> bob:50519296812]}

In [124]:
x.remote_get()

(Wrapper)>[PointerTensor | me:76388321768 -> alice:69575318925]

In [125]:
bob._objects

{}

In [126]:
alice._objects

{39296539549: tensor([1, 2, 3, 4, 5]), 69575318925: tensor([1, 2, 3, 4, 5])}

In [127]:
x.move(bob)

(Wrapper)>[PointerTensor | me:75207849564 -> bob:76388321768]

In [128]:
x

(Wrapper)>[PointerTensor | me:75207849564 -> bob:76388321768]

In [129]:
bob._objects

{76388321768: tensor([1, 2, 3, 4, 5])}

In [130]:
alice._objects

{39296539549: tensor([1, 2, 3, 4, 5])}